In [5]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
capitalsRDD = sc.textFile("/Users/halilergul/Desktop/master/fall-23_24/datasets-20231023/Capitals.txt")
from haversine import haversine

# I define function to parse the lines to extract the country, city, latitude and longitude
def parse_line(line):
    parts = line.split("\t")
    country = parts[1]
    city = parts[2]
    lat, lon = parts[3].split(","), parts[4].split(",")
    return (country, city, float(lat[0]) + float(lat[1])/60, float(lon[0]) + float(lon[1])/60)  # convert minutes to degrees

def correct_swapped_coords(city): # I learned some cities have swapped lat-long values :)
    name, country, lat, lon = city
    # If the latitude is out of range but longitude is in range, they might be swapped
    if (lat > 90 or lat < -90) and (-180 <= lon <= 180):
        lat, lon = lon, lat  # Swap them
    return (name, country, lat, lon)

# After parsing, I correct any swapped lat-long values
cities = capitalsRDD.map(parse_line).map(correct_swapped_coords)
# valid_cities = cities.filter(lambda city: -90 <= city[2] <= 90)
city_pairs = cities.cartesian(cities).filter(lambda x: x[0][1] != x[1][1]) # city pairs RDD

def distance(pair):
    city1, city2 = pair
    coord1 = (city1[2], city1[3]) # (lat1, lon1)
    coord2 = (city2[2], city2[3]) # (lat2, lon2)
    dist = haversine(coord1, coord2)
    return ((city1[1], city2[1]), dist)

distances = city_pairs.map(distance)
farthest_cities = distances.reduce(lambda x, y: x if x[1] > y[1] else y)
print(f"I found {farthest_cities[0][0]} and {farthest_cities[0][1]}")


I found Paramaribo  and Dili 
